In [1]:
import numpy as np
import pandas as pd
import h5py
import os
import json
from sklearn.preprocessing import LabelBinarizer
from functools import partial
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.datasets import make_classification
#from collections import Counter

In [10]:
! head data2.csv

ID,FID,Dx,L
200001,A0001,59118001,7500
200002,A0002,426783006,5000
200003,A0003,164889003,5000
200004,A0004,164889003,5974
200005,A0005,164884008,12500
200006,A0006,59118001,7000
200007,A0007,164889003,5000
200008,A0008,429622005,7588
200009,A0009,164889003,8000


In [11]:
df = pd.read_csv('data2.csv')

In [3]:
#df[df['L']==min(df['L'])]
columns = df.columns
columns
#df[columns[0]]

Index(['ID', 'FID', 'Dx', 'L'], dtype='object')

In [4]:
df['ID'].max()


265344

In [5]:
category = pd.read_table(
    "http://172.16.150.153/liuhui/ecg-public/raw/master/SNOMED_CT-ECG.org",
    sep='\s*[|]\s*', engine='python').dropna(axis=1, how='all')

classes = category.code[:27].to_numpy()

In [75]:
category[category['code']==427172004]

,code,abbreviation,desc,desc_zh,ecgcode
13,427172004,PVC,premature ventricular contractions,室性早搏,0x050202


In [78]:
category[:27]

,code,abbreviation,desc,desc_zh,ecgcode
0,270492004,IAVB,1st degree av block,I度房室传导阻滞,0x060201
1,164889003,AF,atrial fibrillation,心房颤动,0x030701
2,164890007,AFL,atrial flutter,心房扑动,0x030601
3,426627000,Brady,bradycardia,心动过缓,NaN
4,713427006,CRBBB,complete right bundle branch block,完全性右束支传导阻滞,0x060305
5,713426002,IRBBB,incomplete right bundle branch block,不完全性右束支传导阻滞,0x060309
6,445118002,LAnFB,left anterior fascicular block,左前分支阻滞,0x06030e
7,39732003,LAD,left axis deviation,电轴左偏,0x0b0301
8,164909002,LBBB,left bundle branch block,完全性左束支传导阻滞,0x060301
9,251146004,LQRSV,low qrs voltages,QRS电压降低,NaN


In [6]:
list(category[:27]['abbreviation'])

['IAVB',
 'AF',
 'AFL',
 'Brady',
 'CRBBB',
 'IRBBB',
 'LAnFB',
 'LAD',
 'LBBB',
 'LQRSV',
 'NSIVCB',
 'PR',
 'PAC',
 'PVC',
 'LPR',
 'LQT',
 'QAb',
 'RAD',
 'RBBB',
 'SA',
 'SB',
 'NSR',
 'STach',
 'SVPB',
 'TAb',
 'TInv',
 'VPB']

In [7]:
classes
transdict = {}
for i,cato in enumerate(classes):
    transdict[cato] = i
transdict


{270492004: 0,
 164889003: 1,
 164890007: 2,
 426627000: 3,
 713427006: 4,
 713426002: 5,
 445118002: 6,
 39732003: 7,
 164909002: 8,
 251146004: 9,
 698252002: 10,
 10370003: 11,
 284470004: 12,
 427172004: 13,
 164947007: 14,
 111975006: 15,
 164917005: 16,
 47665007: 17,
 59118001: 18,
 427393009: 19,
 426177001: 20,
 426783006: 21,
 427084000: 22,
 63593006: 23,
 164934002: 24,
 59931005: 25,
 17338001: 26}

In [17]:
classes
transdict = {}
for i,cato in enumerate(classes):
    transdict[cato] = i
transdict

{270492004: 0,
 164889003: 1,
 164890007: 2,
 426627000: 3,
 713427006: 4,
 713426002: 5,
 445118002: 6,
 39732003: 7,
 164909002: 8,
 251146004: 9,
 698252002: 10,
 10370003: 11,
 284470004: 12,
 427172004: 13,
 164947007: 14,
 111975006: 15,
 164917005: 16,
 47665007: 17,
 59118001: 18,
 427393009: 19,
 426177001: 20,
 426783006: 21,
 427084000: 22,
 63593006: 23,
 164934002: 24,
 59931005: 25,
 17338001: 26}

In [18]:
def in_27_classes(x):
    for c in x.split(','):
        if int(c) in classes:
            return True
    else:
        return False

In [88]:
def padis(x)
    new = np.zeros((x.shape[0],x.shape[0]))
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            new[i][j] = np.linalg.norm(x[i]-x[j])
    #D=np.linalg.norm(x)
    return new

In [63]:
a = (os.listdir('upsampdata2'))
len(a)
#f = lambda x: 'npy' not in x
#aa=list(map(f,a))
#original max index 34595


34116

In [157]:
train_id.extend(np.arange(47282))
with open("indextrainup.txt", "w") as fp:
    json.dump(list(map(int,train_id)), fp)

In [122]:
for x in a:
    if x == '34596.npy':
        print(x)

In [15]:
a = os.listdir('./upsampdata2')
b = sorted(a, key=lambda x: int(x.split('.')[0]) if (x[1] != 'y' and x[2] != 'y') else 0)

In [182]:
y.shape

(77283, 28)

In [17]:
[i for i, x in enumerate(aa) if x]

[17273, 25650]

In [22]:


keys = []
for keyn in counter.keys():
    if counter[keyn]<3000:
        keys.append(keyn)


24

In [183]:
counter_up = {}
for i in range(1,28):
    counter_up[i-1] = int(sum(y[:,i]))
pos_wts = []
for i in range(27):
    pos_wts.append((77283-list(counter_up.values())[i] )/list(counter_up.values())[i] ) 

In [195]:
pos_wts

[9.481893394818934,
 7.078080903104421,
 79.83995815899581,
 124.05339805825243,
 32.80708661417323,
 14.416517055655296,
 10.748707813925206,
 3.0728853754940713,
 23.706841432225065,
 42.83607487237663,
 22.98603351955307,
 123.8513731825525,
 14.22217845184164,
 39.632492113564666,
 60.776978417266186,
 14.216184288245717,
 18.699974509304106,
 52.70604586518416,
 12.136664966853646,
 21.175896700143472,
 10.224836601307189,
 1.484744236890332,
 9.918762362249224,
 124.25607779578606,
 4.202840985593106,
 17.50646551724138,
 52.93091416608514]

In [9]:
import tensorflow as tf


In [38]:
newkeys=[10, 16, 19, 23]

[10, 16, 19, 23]

In [35]:
keys.extend(np.arange(22, 27))

In [23]:
len(os.listdir('./classify'))

52

In [39]:
def upsampling(x, y, start = 0, key = 0):
    sample_num = np.int(x.shape[0]*1.5)
    choose = np.random.randint(0,x.shape[0],(sample_num))
    index = np.arange(start, start+sample_num)#list(map(int,y[choose,0]))
    y_new = y[choose, :]
    y_new[:,0] = index
    np.save('upsampdata2/'+str(key)+'y_up.npy',y_new) 
    
    for i in range(sample_num):
        idn = index[i]
        x_new = x[choose[i]] + 0.05*np.random.standard_normal((5000,12))
        np.save('upsampdata2/'+str(idn)+'.npy',x_new)
    
      
    #x_new = x[index] + np.random.standard_normal((sample_num,5000,12))
    #ids = y[index, 0]
start = 34596 
for key in newkeys:
    x = np.load('classify/'+str(key)+'.npy')
    y = np.load('classify/'+str(key)+'y.npy')
    
    upsampling(x, y, start, key)
    start += np.int(x.shape[0]*1.5)
    #np.save('upsampdata/class/'+str(key), x_up)



    
    


In [177]:
for key in keys:
    os.remove('upsampdata/'+str(key)+'y_up.npy')


In [43]:
y = train_y
for key in newkeys:
    y = np.vstack ((y,(np.load('upsampdata2/'+str(key)+'y_up.npy'))) )
np.save('train_yup2.npy',y) 

In [55]:
np.save('train_yup2.npy',y) 

In [35]:
sample_num = np.int(x.shape[0]*1.5)
choose = np.random.randint(0,x.shape[0],(sample_num))
y_new = y[choose,:]

In [27]:
y = np.load('classify/'+str(0)+'y.npy')
x = np.load('classify/'+str(0)+'.npy')

In [37]:

index = np.arange(0, 0+sample_num)#list(map(int,y[choose,0]))
y_new[:,0] = index

In [19]:
df = df[df.Dx.apply(in_27_classes)].sample(frac=1, random_state=42).reset_index(drop=True)

In [25]:
df

,ID,FID,Dx,L
0,202138,A2138,59118001,5000
1,250819,HR20819,"164865005,164951009,39732003,426783006",5000
2,245567,HR15567,"164861001,164873001,164951009,39732003,426783006",5000
3,244822,HR14822,"39732003,426783006,445118002",5000
4,260584,E05584,"713426002,428750005",5000
...,...,...,...,...
37614,240423,HR10423,426783006,5000
37615,227791,Q2791,"164867002,164889003,428750005",11500
37616,234801,HR04801,"164861001,164873001,164951009,266249003,397320...",5000
37617,201121,A1121,426783006,5500


In [20]:
print(df.count(),len(df))

str(df.loc[0,['Dx']])
(df)['ID']

ID     37619
FID    37619
Dx     37619
L      37619
dtype: int64 37619


0        202138
1        250819
2        245567
3        244822
4        260584
          ...  
37614    240423
37615    227791
37616    234801
37617    201121
37618    239362
Name: ID, Length: 37619, dtype: int64

In [23]:
df.loc[:30000,['ID', 'Dx']].to_csv('data2/train.csv', index=False)
df.loc[30001:,['ID', 'Dx']].to_csv('data2/test.csv', index=False)

In [11]:
! head test.csv

ID,Dx
239547,426783006
240303,"251146004,426783006"
231439,"164861001,164865005,164873001,164884008,164889003"
250796,426783006
237087,426783006
264027,426783006
259191,426177001
242086,426783006
243376,"164951009,39732003,426783006,713427006,89792004"


In [ ]:
id2indextrain = {}
for i in range(len(trainy['ID'])):
    ID = trainy['ID'][i]
    id2indextrain[ID] = i

In [11]:
train_y=train_y.astype(int)

array([[206821,      0,      0, ...,      0,      0,      0],
       [255326,      1,      0, ...,      0,      0,      0],
       [230875,      0,      0, ...,      0,      0,      0],
       ...,
       [246175,      1,      0, ...,      0,      0,      0],
       [205374,      0,      0, ...,      0,      0,      0],
       [239547,      0,      0, ...,      0,      0,      0]])

In [24]:
def focal_loss_fixed(y_true, y_pred):
    gamma=2.
    alpha=1.
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

    return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1+K.epsilon())) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))


In [25]:
from keras.models import load_model, Sequential
import tensorflow_addons as tfa
import tensorflow.keras.backend as tkb

i = 0
model = load_model('result/0707/wt1focal/multilabel_model_07071'+str(i+5)+str(i)+'.h5',
                   custom_objects={'focal_loss_fixed': focal_loss_fixed})

In [61]:
y_pred = model.predict(test_x)


In [47]:
# transform back to logits
_epsilon = tf.convert_to_tensor(tkb.epsilon(), y_pred1.dtype.base_dtype)
y_pred = tf.clip_by_value(y_pred1, _epsilon, 1 - _epsilon)
y_pred = tf.math.log(y_pred / (1 - y_pred))
pos_wts =[9.481893394818934,7.078080903104421,79.83995815899581,124.05339805825243,32.80708661417323,
          14.416517055655296,10.748707813925206,3.0728853754940713,23.706841432225065,42.83607487237663,
          22.98603351955307,123.8513731825525,14.22217845184164,39.632492113564666,60.776978417266186, 
          14.216184288245717,18.699974509304106,52.70604586518416,12.136664966853646,21.175896700143472, 
          10.224836601307189,1.484744236890332,9.918762362249224,124.25607779578606,4.202840985593106,
          17.50646551724138,52.93091416608514]


In [69]:
i = 0
loss = tf.nn.weighted_cross_entropy_with_logits(labels=y_true[:,i],
                                                    logits=y_pred[:,i],
                                                    pos_weight=pos_wts[i])

In [70]:
loss

<tf.Tensor: shape=(8245,), dtype=float32, numpy=
array([0.69812804, 0.79882586, 0.69447476, ..., 0.69797456, 0.69644254,
       0.70219713], dtype=float32)>

In [67]:
y_pred1 = tf.convert_to_tensor(y_pred, dtype=tf.float32)

In [71]:
test_y = np.load('test_y.npy')[:,1:].astype(np.float32)


In [75]:
test_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [74]:
type(test_y[0,1])

numpy.float32

In [64]:
y_true

<tf.Tensor: shape=(8245, 27), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [72]:
test_y = np.load('data2/test_y.npy')
train_y = np.load('data2/train_y.npy')
len(test_y)+len(train_y)

37619

In [34]:
new = np.zeros((8245,1))
for i in range(len(test_y)):
    new[i,0] = testy.loc[i][0]


In [195]:
#test_y = np.hstack((new,test_y))
test_y[:,1:].shape

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [65]:
np.save('train_y.npy',train_y)

In [56]:
len(os.listdir('ecgtrain'))
len(os.listdir('ecgtest'))

0

In [3]:
import os
for file in os.listdir('result/0719/weightedloss/'):
    print(file)

multilabel_model_0719162.h5
multilabel_model_0719161.h5
multilabel_model_0719164.h5
multilabel_model_0719150.h5
multilabel_model_0719163.h5


In [4]:
os.listdir('result/0719/weightedloss/')

['multilabel_model_0719162.h5',
 'multilabel_model_0719161.h5',
 'multilabel_model_0719164.h5',
 'multilabel_model_0719150.h5',
 'multilabel_model_0719163.h5']

In [51]:
sorted(
        ['2-.npy', '2-.npy', '4-.npy','12.npy', '2-.npy', '21.npy'], key=lambda x: int(x[0]) if x[1] == '-' else int(x[:2]))

['2-.npy', '2-.npy', '2-.npy', '4-.npy', '12.npy', '21.npy']

In [73]:
counter = {}
for i in range(1,28):
    counter[i-1] = int(sum(train_y[:,i]))

In [74]:
counter

{0: 1915,
 1: 2764,
 2: 252,
 3: 219,
 4: 523,
 5: 1283,
 6: 1416,
 7: 4818,
 8: 843,
 9: 454,
 10: 793,
 11: 254,
 12: 1383,
 13: 148,
 14: 269,
 15: 1212,
 16: 807,
 17: 329,
 18: 1916,
 19: 984,
 20: 1903,
 21: 16489,
 22: 1934,
 23: 173,
 24: 3726,
 25: 899,
 26: 282}

In [53]:
np.sum([counter[key] for key in newkeys])*1.5

4111.5

In [57]:
a = np.array( list(counter[key] for key in keys))
a

#np.sum(a[:])

array([1879, 2696,  244,  211,  516, 1234, 1403,  815,  439,  776,  235,
       1373,  689,  259, 1189,  810,  345, 1866,  992, 1845, 1915,  163,
        880,  295])

In [58]:

a = list(map(int, a*1.5))


In [59]:
np.sum(a)

34596

In [12]:
max_val = float(max(counter.values()))  
print(max_val)
class_weight = {class_id : (max_val/num_ecg)**1 for class_id, num_ecg in counter.items()}
class_weight

16213.0


{0: 25.34573582116018,
 1: 14.747392607351602,
 2: 541.6395075252537,
 3: 673.5524048919552,
 4: 176.1248035122267,
 5: 47.6236123553626,
 6: 39.28335462323158,
 7: 6.244777346665236,
 8: 88.72758499006852,
 9: 224.43898381195044,
 10: 95.49980661876873,
 11: 573.0510078045274,
 12: 40.57787203148497,
 13: 114.14745714630708,
 14: 495.2738838571483,
 15: 50.35264538922846,
 16: 89.55040322831849,
 17: 322.15615366930547,
 18: 25.611063618261223,
 19: 66.0735385301596,
 20: 26.049567571343328,
 21: 1.0,
 22: 24.634395326738932,
 23: 992.0045575859164,
 24: 9.258798428276235,
 25: 79.08080213736375,
 26: 407.4384866643187}

In [96]:
(os.path.join('ecgtrain5000', '259668.npy'))

'ecgtrain5000/259668.npy'

In [51]:
test_y[test_y[:,0]==int(test_y[2,0])][0,1:]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
len(trainy)+len(testy)

37619

In [88]:
train_y[:10]

array([[202138,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      1,      0,      0,      0,      0,
             0,      0,      0,      0],
       [250819,      0,      0,      0,      0,      0,      0,      0,
             1,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      1,      0,
             0,      0,      0,      0],
       [245567,      0,      0,      0,      0,      0,      0,      0,
             1,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      1,      0,
             0,      0,      0,      0],
       [244822,      0,      0,      0,      0,      0,      0,      1,
             1,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      1,      0,
             

In [87]:
index=np.hstack((np.arange(19),np.arange(20,24), np.arange(25,27))) 
train_y[:10,index]

array([[202138,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0],
       [250819,      0,      0,      0,      0,      0,      0,      0,
             1,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      1,      0,      0,
             0],
       [245567,      0,      0,      0,      0,      0,      0,      0,
             1,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      1,      0,      0,
             0],
       [244822,      0,      0,      0,      0,      0,      0,      1,
             1,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      1,      0,      0,
             0],
       [260584,      0,      0,      0,      0,      0,      1,     

In [89]:
def transto24(train_y):
    for i in range(train_y.shape[0]):
        if train_y[i, 19]:
            train_y[i, 5] = 1
        if train_y[i, 24]:
            train_y[i, 13] = 1
        if train_y[i, 27]:
            train_y[i, 14] = 1
    index = np.hstack((np.arange(19),np.arange(20,24), np.arange(25,27))) 
    return train_y[:,index]
train_yy = transto24(train_y)
test_yy = transto24(test_y)
np.save('data2/train_y24.npy',train_yy)
np.save('data2/test_y24.npy',test_yy)

In [32]:
#df.loc[:100,['Dx']], generate target matrix
def generate(df):
    new = np.zeros((len(df),len(classes))) #our target matrix
    for i in range(len(df)):
        catolist = df.loc[i][1].split(',')
        for cato in catolist:
            if int(cato) in classes:
                new[i,transdict[int(cato)]] = 1
    return new

def generate24(df):
    new = np.zeros((len(df),24)) #our target matrix
    for i in range(len(df)):
        catolist = df.loc[i][1].split(',')
        for cato in catolist:
            if int(cato) in classes:
                new[i,transdict[int(cato)]] = 1
    return new

trainy = pd.read_csv('data2/train.csv')
testy = pd.read_csv('data2/test.csv')
train_y = generate(trainy)
test_y = generate(testy)
np.save('data2/train_y.npy',train_y)
np.save('data2/test_y.npy',test_y)


In [45]:
indextrain = np.array(indextrain).reshape((train_y.shape[0],1))
indextest = np.array(indextest).reshape((test_y.shape[0],1))

In [69]:
trainy

,ID,Dx
0,206821,426783006
1,255326,"59118001,270492004"
2,230875,"164947007,426783006"
3,228332,"284470004,164865005"
4,255526,"428750005,59931005"
...,...,...
29996,264894,427084000
29997,247843,"164934002,426783006"
29998,246175,"164865005,251146004,270492004,39732003,426783006"
29999,205374,59118001


In [48]:
train_y = np.hstack((indextrain,train_y))
test_y = np.hstack((indextest,test_y))

In [50]:
y = y.astype(int)

In [58]:
np.save('data2/test_y.npy',test_y)


In [57]:
test_y

array([[249363.,      0.,      0., ...,      0.,      0.,      0.],
       [261220.,      0.,      0., ...,      0.,      0.,      0.],
       [244008.,      0.,      0., ...,      0.,      0.,      0.],
       ...,
       [234801.,      0.,      0., ...,      0.,      0.,      0.],
       [201121.,      0.,      0., ...,      0.,      0.,      0.],
       [239362.,      0.,      0., ...,      0.,      0.,      0.]])

In [37]:
def read_ecg_record_and_info(N, root='/ecgdb'):
    subdir = '{:03d}'.format((N - 100000) // 5000 + 1)
    h5 = '{}/{}/{}.h5'.format(root, subdir, N)
    with h5py.File(h5, 'r') as f:
        sig = f['signal'][()]
        info = dict(f.attrs)
    return sig, info

In [14]:
sig, other = read_ecg_record_and_info(240233)

In [12]:
other = pd.read_csv('other.csv')
other.iloc[:5,1]

0    49.0
1    56.0
2    77.0
3    65.0
4    77.0
Name: age, dtype: float64

In [8]:
with open("indextrain.txt", "r") as fp:
    train_id = json.load(fp)
with open("indextest.txt", "r") as fp:
    test_id = json.load(fp)
    
    

In [14]:
x = np.load('ecgtrain/' + str(train_id[0]) + '.npy')
x.shape

(5000, 12)

In [69]:
agetest = []
for ID in test_id:
    agetest.append(other[other['ID']==ID]['age'])
agetest = np.array(agetest)
    

In [84]:
import collections
def dataresamp():
    dataresampling = collections.defaultdict(list)
    dataresamplingy = collections.defaultdict(list)
    for ID in train_id:
        x = np.load('ecgtrain/' + str(ID) + '.npy')
        y = train_y[train_y[:,0]==ID][0,:] #obtain class index
        for cato in [7,24]:#range(1,28):
            if y[cato+1]==1:
                dataresampling[cato].append(x)
                dataresamplingy[cato].append(y)
    return dataresampling, dataresamplingy
    
    

In [85]:
dataresampling, dataresamplingy = dataresamp()

In [130]:
for key in newkeys:
    np.save('classify/'+str(key)+'.npy', np.array(dataresampling[key]))
for key in newkeys:
    np.save('classify/'+str(key)+'y.npy', np.array(dataresamplingy[key]))

In [143]:
np.array(dataresamplingy[20]).shape

(992, 28)

In [152]:
xx = np.load('classify/'+str(0)+'.npy' )
xx.shape

(1879, 5000, 12)

In [31]:
for key in keys:
    np.save('classify/'+str(key)+'y.npy', np.array(dataresamplingy[i]))

In [ ]:
np.save('class/'+str(0)+'.npy', np.array(dataresampling[i]))

In [ ]:
for i in range(27): 
    dataresampling[i] = np.array(dataresampling[i])

In [86]:
with open("presamplingx"+".json", "w") as fp:
    json.dump(dataresampling, fp)

TypeError: Object of type 'ndarray' is not JSON serializable

In [ ]:
#find class name 
class_name = transdict
cnt = 0
for ID in train_id:
    _, other = read_ecg_record_and_info(ID)
    other

In [107]:
len(df)==len(other)

True

In [134]:
cnt1 = 0 #tr
cnt2 = 0 #te
ageminmaxfinal = []

for i in range(len(df)):
    ID = other['ID'][i]
    if cnt1<len(train_id) and train_id[cnt1]==ID:
        ageminmaxfinal.append(ageminmax[cnt1][0])
        cnt1+=1
    if test_id[cnt2]==ID:
        ageminmaxfinal.append(ageteminmax[cnt2][0])
        cnt2+=1
    
    
    

In [139]:
min(ageminmaxfinal)

0.0

In [125]:
lb = LabelBinarizer()
lb.fit(other['sex'])
lb.classes_
sexoh = lb.transform(other['sex'])[:,0]
sexoh

array([0, 1, 0, ..., 0, 0, 1])

In [133]:
ageteminmax[0][0]

0.7216494845360825

In [140]:

otherprocess = pd.DataFrame({'ID':other['ID'], 'age':ageminmaxfinal, 'sex':sexoh})

In [141]:
otherprocess

,ID,age,sex
0,206821,0.494845,0
1,255326,0.567010,1
2,230875,0.783505,0
3,228332,0.659794,1
4,255526,0.783505,1
...,...,...,...
38241,226026,0.804124,1
38242,234170,0.845361,1
38243,265243,0.391753,0
38244,200990,0.618557,0


In [142]:
otherprocess.to_csv('otherprocess.csv', index=False)

In [166]:
X2 = np.empty((64, 2))
X2[1] = otherprocess[otherprocess['ID']==200990].iloc[0,1:]

In [167]:
X2[1]

array([0.6185567, 0.       ])

In [37]:
agetrain = np.array(agetrain)
len(agetrain[~np.isnan(agetrain)]  )+len(agetrain[np.isnan(agetrain)]  )

In [92]:
print(max(agetest),min(agetest) )
np.argwhere(agetest<0)[0,0]

[104.] [-1.]


4284

In [77]:
ageminmax = (agetrain-min(agetrain))/(max(agetrain)-min(agetrain))

In [98]:
ageteminmax = (agetest-min(agetrain))/(max(agetrain)-min(agetrain))

In [100]:
min(ageteminmax)

array([0.03092784])

In [79]:
len(agetrain[agetrain>0])

30001

In [76]:
agetrain[13420] = int(agetrain[agetrain>0].mean())
agetrain[19966] = int(agetrain[agetrain>0].mean())
agetrain[np.isnan(agetrain)] = 60

In [95]:
agetest[np.argwhere(agetest<0)[0,0]] = int(agetest[agetest>0].mean())
agetest[np.isnan(agetest)] =  int(agetest[agetest>0].mean())

In [91]:
int(agetest[agetest>0].mean())

60

In [97]:
np.sum((agetest)<0)

0

In [14]:
other['sex']

0        F
1        M
2        F
3        M
4        M
        ..
38241    M
38242    M
38243    F
38244    F
38245    M
Name: sex, Length: 38246, dtype: object

In [31]:
agenalist = list(other.loc[pd.isna(other["age"]), :].index)
len(agenalist)


163

In [22]:
print(sig.shape, other)
sig[0][0]=None
np.sum(np.isnan(sig))


(12, 2500) {'Dx': '426783006', 'age': 50.0, 'fid': 'E09923', 'fs': 500, 'n_sig': 12, 'sex': 'F', 'sig_len': 2500, 'sig_name': array(['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4',
       'V5', 'V6'], dtype=object)}


1

In [34]:
indexna=[]
def checkna(ID):
    sig, _ = read_ecg_record_and_info(ID)
    if np.sum(np.isnan(sig))>0:
        indexna.append(ID)
    

for i in range(len(df)):
    ID = df['ID'][i]
    checkna(ID)


In [21]:
indexna=[259603, 261072, 261909, 262675, 262941, 263321]

In [22]:
#deal with NAN value and assign 0
indexna
for index in indexna:
    sig, _ = read_ecg_record_and_info(index)
    sig[np.isnan(sig)] = 0
    sig = sig[:,:5000]
    np.save('ecgtrain/'+str(index)+'.npy',np.transpose(sig,(1,0)))

In [70]:
indexna

[259603, 261072, 261909, 262675, 262941, 263321]

In [34]:
IDs = []
age = []
sex = []
for i in range(len(df)):
    ID = df['ID'][i]
    _, other = read_ecg_record_and_info(ID)
    IDs.append(ID)
    age.append(other['age'])
    sex.append(other['sex'])
other = pd.DataFrame({'ID':IDs, 'age':age, 'sex':sex})



In [78]:
agetr = []
sextr = []
agete = []
sexte = []
for i in range(len(train_id)):
    ID = train_id[i]
    agetr.append(other[other['ID']==ID]['age'][i])
    sextr.append(other[other['ID']==ID]['sex'][i])
for i in range(len(test_id)):
    ID = test_id[i]
    agete.append(other[other['ID']==ID]['age'][i])
    sexte.append(other[other['ID']==ID]['sex'][i])
othertr = np.zeros((len(indextrain),3))   #age,female,male
otherte = np.zeros((len(indextest),3))   #age,female,male


    

KeyError: 0

In [61]:
sextr

['F']

In [52]:
other[other['ID']==206821]['sex'][0]

'F'

In [41]:
np.argwhere(np.isnan(sig))
sig[np.isnan(sig)] = 0
np.sum(np.isnan(sig))

0

In [61]:
with open("data2/indextrain.txt", "w") as fp:
    json.dump(list(map(int,indextrain)), fp)
with open("data2/indextest.txt", "w") as fp:
    json.dump(list(map(int,indextest)), fp)

In [71]:
with open("indextrain.txt", "w") as fp:
    json.dump(list(map(int,indextrain)), fp)

In [66]:
len(indextest)

7618

In [41]:
with open("indexna.txt", "w") as fp:
    json.dump(list(map(int,indexna)), fp)

In [38]:
indextrain = []
for i in range(len(trainy)):    
    #sig, _ = read_ecg_record_and_info(trainy.loc[i][0])
    #if sig.shape[1]>=5000:
    indextrain.append(int(trainy.loc[i][0]))
indextest = []           
for i in range(len(testy)):    
    #sig, _ = read_ecg_record_and_info(testy.loc[i][0])
    #if sig.shape[1]>=5000:
    indextest.append(int(testy.loc[i][0]))
        

In [40]:
len(indextrain)+len(indextest)

37619

In [103]:
new = np.zeros((64,5001,12))
for i in range(len(df)):
    ID = df['ID'][i]
    sig, other = read_ecg_record_and_info(ID)
    new[i,:5000,:] = sig
    

[206821,
 255326,
 230875,
 228332,
 255526,
 232393,
 201557,
 204710,
 239190,
 200546]

In [67]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,shuffle=True,random_state=1)
fivefdtridx = []
fivefdvalidx = []
for i,(train_index,test_index) in enumerate(kf.split(indextrain)):
    trainid = np.array(indextrain)[train_index]
    valid = np.array(indextrain)[test_index]
    fivefdtridx.append(trainid)
    fivefdvalidx.append(valid)
fivedictr = {}
fivedicval = {}
for i in range(5):
    fivedictr[i] = list(map(int,fivefdtridx[i]))
    fivedicval[i] = list(map(int,fivefdvalidx[i]))
with open('data2/fivefdtrid.json', 'w') as file:
    json.dump(fivedictr, file)
with open('data2/fivefdvalid.json', 'w') as file:
    json.dump(fivedicval, file)

In [57]:
'0713/multilossmpl.1/'
print(len(indextest),len(indextrain),len(indextrain)+len(indextest))

8245 30001 38246


In [19]:
'result/'+'0713/multilossmpl.1'


'result/0713/multilossmpl.1'

In [15]:
with open("indextrain.txt", "r") as fp:
    train_id = json.load(fp)
with open("indextest.txt", "r") as fp:
    test_id = json.load(fp)
train_y = np.load('train_y.npy')
test_y = np.load('test_y.npy')
test_x = []
test_yy = []
for ID in test_id:
    test_x.append(np.load('ecgtest/' + str(ID) + '.npy'))
    test_yy.append(test_y[test_y[:,0]==ID][0,1:]) 

test_x = np.array(test_x)
test_y = np.array(test_yy)

In [28]:
test_y.shape

(8245, 27)

In [31]:
from imblearn.over_sampling import SMOTE
ros = SMOTE(random_state=0)

X_ros, y_ros = ros.fit_resample(test_x, test_y)


ValueError: Imbalanced-learn currently supports binary, multiclass and binarized encoded multiclasss targets. Multilabel and multioutput targets are not supported.

In [176]:
import keras.backend as K
K.image_data_format()

'channels_last'

In [182]:
with open("data2/indextrain.txt", "w") as fp:
    json.dump(indextrain, fp)

with open("data2/indextest.txt", "w") as fp:
    json.dump(indextest, fp)

In [120]:
len(test_id)

8233

In [112]:
trainy.loc[1][0]

test_x = np.zeros((len(testy),12,max(df['L'])))

In [100]:
print(len(trainy),len(os.listdir('ecgtest5000')),len(os.listdir('ecgtrain5000')))
print(len(os.listdir('ecgtrain')))

30001 8233 29959
30001


In [20]:
i = 0
sig, _ = read_ecg_record_and_info(trainy.loc[i][0])
print(sig.shape)
#train_x[i,:,:sig.shape[1]] = sig
print(sig[:,:2],sig[:,:2].shape)
print(np.transpose(sig,(1,0)).shape,np.transpose(sig,(1,0))[:2,:])

(12, 7500)
[[ 0.24468821  0.22968851]
 [ 0.22841503  0.21742846]
 [-0.01627641 -0.01227279]
 [-0.23648237 -0.22348018]
 [ 0.12939914  0.12039987]
 [ 0.10486002  0.10186549]
 [-0.01836879 -0.01436347]
 [ 0.3780261   0.35602352]
 [ 0.27770615  0.26073414]
 [ 0.26451504  0.24753527]
 [ 0.29356524  0.27658698]
 [ 0.24786747  0.23485386]] (12, 2)
(7500, 12) [[ 0.24468821  0.22841503 -0.01627641 -0.23648237  0.12939914  0.10486002
  -0.01836879  0.3780261   0.27770615  0.26451504  0.29356524  0.24786747]
 [ 0.22968851  0.21742846 -0.01227279 -0.22348018  0.12039987  0.10186549
  -0.01436347  0.35602352  0.26073414  0.24753527  0.27658698  0.23485386]]


In [21]:
def genex(trainy):
    train_x = np.zeros((len(trainy),12,max(df['L'])))
    for i in range(len(trainy)):
        sig, _ = read_ecg_record_and_info(trainy.loc[i][0])
        train_x[i,:,:sig.shape[1]] = sig
    return train_x
#train_x = genex(trainy)
#test_x = genex(testy)

In [33]:
trainy.loc[0][0]

206821

In [38]:
'ecgtrain'+str(trainy.loc[0][0])+'.npy'

'ecgtrain206821.npy'

In [96]:
def savex(trainy):
    for i in range(len(trainy)):
        tmp = np.zeros((5000,12))
        sig, _ = read_ecg_record_and_info(trainy.loc[i][0])
        sig = np.transpose(sig,(1,0)) #(5000,12)
        
        if sig.shape[0]>5000:
            start_inx = np.random.randint(0,sig.shape[0]-4999,size=1)[0]
            sig = sig[start_inx:(start_inx+5000),:]
        tmp[:sig.shape[0],] = sig
        np.save('ecgtrainnew5000/'+str(trainy.loc[i][0])+'.npy',tmp)
def savex5000(trainy):
    for i in range(len(trainy)):
        sig, _ = read_ecg_record_and_info(trainy.loc[i][0])
        if sig.shape[1]<5000:
            continue
        sig = sig[:,:5000]
        np.save('ecgtest5000/'+str(trainy.loc[i][0])+'.npy',np.transpose(sig,(1,0)))


In [94]:
i=0
tmp = np.zeros((5000,12))
sig, _ = read_ecg_record_and_info(trainy.loc[i][0])
sig = np.transpose(sig,(1,0)) #(5000,12)
sig[2:(2+5000),:].shape

(5000, 12)

In [97]:
savex(trainy)

In [17]:
trainy = pd.read_csv('train.csv')
testy = pd.read_csv('test.csv')
#train_x = np.load('train_x.npy')
#test_x = genex(testy)
#train_x.shape

In [11]:
test_x.shape

(8246, 12, 72000)

In [ ]:
#np.save('train_x.npy',train_x)
np.save('test_x.npy',test_x)